# Crashes to JSON

Use `Data Query Tool` to create a new export from UCONN Crashes (Town=Hartford, last five years): https://www.ctcrash.uconn.edu/QueryTool2.action

This notebook combines data from two tables (0 and 2) into a small JSON file, which is an array of the following objects:
```
{
    "x":41.73936,
    "y":-72.66404,
    "d":1593820800,
    "p":0,
    "c":0
}
```

* `d` is timestamp that needs to be multiplied by `100,000` for a proper JS timestamp
* `p` is a flag for pedestrians (1=true, 0=false)
* `c` is a flag for cyclists
* `id` is crash ID
* `r` is route class (1-Interstate, 2-USRoute, 3-State, 4-Local)
* `s` is crash severity (O for no injuries, A for any injury, and K for fatal crash)

In [28]:
import pandas as pd

In [29]:
export = '70836'

### Import roles from table 2

In [30]:
roles = pd.read_csv(
    f'export_{export}/export_{export}_2.csv',
    skiprows=1,
    usecols=['CrashId', 'Person Type Text Format']
)

roles = roles.groupby(['CrashId'])['Person Type Text Format'].unique()

### Import crashes from table 0

In [31]:
crashes = pd.read_csv(
    f'export_{export}/export_{export}_0.csv',
    skiprows=1,
    usecols=['CrashId', 'Latitude', 'Longitude', 'Date Of Crash', 'Time of Crash', 'Crash Severity', 'Route Class']
)

### Merge data and save to .json

In [32]:
crashes.columns = ['id', 'x', 'y', 'd', 't', 's', 'r']

crashes.d = crashes.d.apply(lambda x: int(pd.to_datetime(x).timestamp() / 100))
crashes.t = crashes.t.apply(lambda x: x[:-3])

crashes['p'] = crashes.id.apply(lambda x: 1 if x in roles and 'Pedestrian' in roles[x] else 0)
crashes['c'] = crashes.id.apply(lambda x: 1 if x in roles and 'Bicyclist' in roles[x] else 0)

In [33]:
crashes.to_json('data/crashes.json', orient='records', double_precision=5)

In [49]:
pd.to_datetime('2015-01-01').timestamp()
#pd.to_datetime('2020-07-31').timestamp()

1420070400.0

In [50]:
crashes[ (crashes.d >= 14200704) & (crashes.d <= 15961536) ]

,id,x,y,d,t,s,r,p,c
0,26165,41.736755,-72.706648,14202432,15:11,O,4,0,0
1,26166,41.784846,-72.708589,14203296,14:27,O,4,0,0
2,26168,41.793270,-72.694880,14217984,02:30,O,4,0,0
3,26172,41.752365,-72.685040,14201568,08:23,O,4,0,0
4,26173,41.774810,-72.693833,14209344,18:57,A,4,1,0
...,...,...,...,...,...,...,...,...,...
38161,788385,41.756281,-72.682520,15938208,20:00,A,4,0,0
38162,788386,41.799010,-72.662266,15938208,20:52,A,4,0,0
38163,788387,41.760758,-72.706405,15938208,21:49,O,4,0,0
38164,788428,41.771024,-72.670760,14210208,08:59,O,1,0,0
